In [1]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
import pickle

In [2]:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [3]:
# Read data from files
train = pd.read_csv( os.path.join(DATADIR, 'labeledTrainData.tsv'), header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( os.path.join(DATADIR, "unlabeledTrainData.tsv"), header=0,  delimiter="\t", quoting=3 )

In [4]:
# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews " \
     "and %d unlabeled reviews\n" % (train["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews and 50000 unlabeled reviews



In [5]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [6]:
print train.ix[1,][2]
train.loc[0:2]

"\"The Classic War of the Worlds\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \"critics\" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \"critics\". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \"critics\" perceive to be its shortcomings."


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


### Get an even split train/test set from our labelled training set

because Kaggle's test set has no labels so we need our own labelled test set.

In [7]:
fracTrain = 0.5
nSamples = train.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = train.ix[order[:splitIndex],:]
test1 = train.ix[order[splitIndex:],:]

In [8]:
print train1.shape
print test1.shape

(12500, 3)
(12500, 3)


In [9]:
print test1.iloc[[0]]

             id  sentiment                                             review
14772  "585_10"          1  "Upon seeing this film once again it appeared ...


In [28]:
pickle.dump(train1, open('./pickled_objects/train1.p', 'wb'))
pickle.dump(test1, open('./pickled_objects/test1.p', 'wb'))
pickle.dump(unlabeled_train, open('./pickled_objects/unlabeled_train.p', 'wb'))

# Word2Vec inversion

Here is some stuff to split the labelled training set into positive and negative reviews. NOTE: takes a long time to run.

Gets a list of positive review indexes and negative review indexes. Takes those respective reviews and parses them into sentences and appends to a list of positive and negative sentences, respectively. Also does the same for the unlabelled training sentences.

In [11]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences
sentences_unlabelled = []  # Initialize an empty list of sentences

inxs_pos = np.where(train1["sentiment"] == 1)[0].tolist()
inxs_neg = np.where(train1["sentiment"] == 0)[0].tolist()

print "Parsing sentences from training set"
for inx in inxs_pos:
    review = train1["review"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["review"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    
    
print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences_unlabelled += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)


Parsing sentences from unlabeled set


/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup

Now do the same for all sentences without splitting to build a vocabulary:

In [12]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train1["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentences from unlabeled set


Some stuff from Taddy's code (much simpler than Kaggle method for building Word2Vec models). 

In [13]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [14]:
basemodel.build_vocab(sentences) 

Train 2 models off of the base model (one pos & one neg).

In [15]:
from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

3353066

In [29]:
pickle.dump(models, open('./pickled_objects/models.p', 'wb'))

### Inversion of the distributed representations

At this point, we have 5 different word2vec language representations. Each 'model' has been trained conditional (i.e., limited to) text from a specific star rating. We will apply Bayes rule to go from p(text|stars) to p(stars|text).
For any new sentence we can obtain its likelihood (lhd; actually, the composite likelihood approximation; see the paper) using the score function in the word2vec class. We get the likelihood for each sentence in the first test review, then convert to a probability over star ratings. Every sentence in the review is evaluated separately and the final star rating of the review is an average vote of all the sentences. This is all in the following handy wrapper.

In [31]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [30]:
# read in the test set as a list of a list of words
docs = []
for review in test1["review"]:
    docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


In [32]:
print docs[0]

[[u'upon', u'seeing', u'this', u'film', u'once', u'again', u'it', u'appeared', u'infinitely', u'superior', u'to', u'me', u'this', u'time', u'than', u'the', u'previous', u'times', u'i', u'have', u'viewed', u'it'], [u'the', u'acting', u'is', u'stunningly', u'wonderful'], [u'the', u'characters', u'are', u'very', u'clearly', u'drawn'], [u'brad', u'pitt', u'is', u'simply', u'superb', u'as', u'the', u'errant', u'son', u'who', u'rebels'], [u'the', u'other', u'actors', u'and', u'actresses', u'are', u'equally', u'fine', u'in', u'every', u'respect'], [u'robert', u'redford', u'creates', u'a', u'wonderful', u'period', u'piece', u'from', u'the', u'days', u'of', u'speakeasies', u'of', u'the', u's'], [u'the', u'scenery', u'is', u'incredibly', u'beautiful', u'of', u'the', u'mountains', u'and', u'streams', u'of', u'western', u'montana'], [u'all', u'in', u'all', u'this', u'is', u'one', u'of', u'the', u'finest', u'films', u'made', u'in', u'the', u's', u'you', u'must', u'see', u'this', u'movie']]


In [33]:
probstest = docprob(docs[0:2], models)

In [34]:
probstest

,0,1
doc,,
0,0.066164,0.933836
1,0.247392,0.752608


### Test set example
As an example, we apply the inversion on the full test set.

In [35]:
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,models).astype(object)

In [36]:
probs

,0,1
doc,,
0,0.0661639,0.933836
1,0.247392,0.752608
2,0.238121,0.761879
3,0.643324,0.356676
4,0.730128,0.269872
5,0.409516,0.590484
6,0.803493,0.196507
7,0.448627,0.551373
8,0.124598,0.875402


In [37]:
pickle.dump(probs, open('./pickled_objects/probs.p', 'wb'))
pickle.dump(docs, open('./pickled_objects/docs.p','wb'))

A numpy array of ones for the number of docs that we're predicting:

In [38]:
predictions = np.ones((probs.shape[0]))

In [39]:
print np.shape(predictions)

(12500,)


In [40]:
predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

In [41]:
predictions

array([ 1.,  1.,  1., ...,  0.,  1.,  1.])

In [42]:
print predictions.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [44]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(test1["sentiment"], predictions)

0.867806437407


In [45]:
pickle.dump(predictions, open('./pickled_objects/predictions.p', 'wb'))

### probsentences was created by commenting out the "prob = prob.groupby("doc").mean()" line in the function and then getting probs.

In [46]:
probssentences = docprobsentences(docs,models).astype(object)

In [47]:
pickle.dump(probssentences, open('./pickled_objects/probssentences.p', 'wb'))

In [48]:
probssentences[0:20]

,0,1,doc
0,0.00338731,0.996613,0
1,0.0392682,0.960732,0
2,0.412265,0.587735,0
3,0.00132289,0.998677,0
4,0.0169766,0.983023,0
5,0.034818,0.965182,0
6,0.0208975,0.979103,0
7,0.000375714,0.999624,0
8,1.52217e-05,0.999985,1
9,3.9868e-07,1,1


## Need to get sentence probabilities for the training set

In [21]:
"""
This function generates probssentences (probabilities for each sentences before the mean is taken.
"""

def docprobsentences(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    #prob = prob.groupby("doc").mean()
    return prob

In [19]:
# read in the test set as a list of a list of words
docs_train = []
for review in train1["review"]:
    docs_train.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

In [22]:
probssentences_train = docprobsentences(docs_train,models).astype(object)

In [23]:
probssentences_train

,0,1,doc
0,0.936547,0.0634529,0
1,0.911208,0.0887919,0
2,0.987688,0.012312,0
3,0.294557,0.705443,0
4,0.1607,0.8393,0
5,0.999999,1.17797e-06,0
6,0.820259,0.179741,0
7,0.97032,0.0296802,0
8,0.985021,0.0149791,0
9,0.992383,0.00761658,0


In [27]:
pickle.dump(probssentences_train, open('./pickled_objects/probssentences_train.p', 'wb'))